In [341]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [342]:
df = pd.read_csv('thyroidDF.csv')
df.head()

age sex on_thyroxine query_on_thyroxine on_antithyroid_meds sick pregnant  \
0   29   F            f                  f                   f    f        f   
1   29   F            f                  f                   f    f        f   
2   41   F            f                  f                   f    f        f   
3   36   F            f                  f                   f    f        f   
4   32   F            f                  f                   f    f        f   

  thyroid_surgery I131_treatment query_hypothyroid  ...    TT4 T4U_measured  \
0               f              f                 t  ...    NaN            f   
1               f              f                 f  ...  128.0            f   
2               f              f                 f  ...    NaN            f   
3               f              f                 f  ...    NaN            f   
4               f              f                 f  ...    NaN            f   

  T4U FTI_measured FTI TBG_measured   TBG  referral_source target  patient_id  
0 NaN            f NaN            f   NaN            other      -   840801013  
1 NaN            f NaN            f   NaN            other      -   840801014  
2 NaN            f NaN            t  11.0            other      -   840801042  
3 NaN            f NaN            t  26.0            other      -   840803046  
4 NaN            f NaN            t  36.0            other      S   840803047  

[5 rows x 31 columns]

# A. DATA PREPROCESSING

In [347]:
diagnosis = {'-': 'negative',
             'A': 'hyperthyroid', 
             'B': 'hyperthyroid', 
             'C': 'hyperthyroid', 
             'D': 'hyperthyroid',
             'E': 'hypothyroid', 
             'F': 'hypothyroid', 
             'G': 'hypothyroid', 
             'H': 'hypothyroid'}
df['target'] = df['target'].map(diagnosis)


In [350]:
df['target'].head()

0    negative
1    negative
2    negative
3    negative
4         NaN
Name: target, dtype: object

Mendapatkan informasi tentang data,

In [351]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  9172 non-null   int64  
 1   sex                  8865 non-null   object 
 2   on_thyroxine         9172 non-null   object 
 3   query_on_thyroxine   9172 non-null   object 
 4   on_antithyroid_meds  9172 non-null   object 
 5   sick                 9172 non-null   object 
 6   pregnant             9172 non-null   object 
 7   thyroid_surgery      9172 non-null   object 
 8   I131_treatment       9172 non-null   object 
 9   query_hypothyroid    9172 non-null   object 
 10  query_hyperthyroid   9172 non-null   object 
 11  lithium              9172 non-null   object 
 12  goitre               9172 non-null   object 
 13  tumor                9172 non-null   object 
 14  hypopituitary        9172 non-null   object 
 15  psych                9172 non-null   o

pada datasets terdapat data redudant, terdapat 7 kolom redundant, yaitu array dengan npmoe 16 TSH_measured, 18 T3_measured, 20 T4_measured 22 T4U_measured, 24 FTI_measured, 26 TBG_measured, refral_score. 

Selanjutnya akan kami hapus kolom redundant, dan menghapus kolom id pasien 

In [352]:
df.drop([
    'TSH_measured',
    'T3_measured',
    'TT4_measured',
    'T4U_measured',
    'FTI_measured',
    'TBG_measured',
    'referral_source',
    'patient_id'
], axis=1, inplace=True)

In [353]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  9172 non-null   int64  
 1   sex                  8865 non-null   object 
 2   on_thyroxine         9172 non-null   object 
 3   query_on_thyroxine   9172 non-null   object 
 4   on_antithyroid_meds  9172 non-null   object 
 5   sick                 9172 non-null   object 
 6   pregnant             9172 non-null   object 
 7   thyroid_surgery      9172 non-null   object 
 8   I131_treatment       9172 non-null   object 
 9   query_hypothyroid    9172 non-null   object 
 10  query_hyperthyroid   9172 non-null   object 
 11  lithium              9172 non-null   object 
 12  goitre               9172 non-null   object 
 13  tumor                9172 non-null   object 
 14  hypopituitary        9172 non-null   object 
 15  psych                9172 non-null   o

melakukan pengecekan nilai usia yang lebih dari 100, nilai lebih dari 100 tahun akan tidak digunakan

In [354]:
df[df.age > 100]

age sex on_thyroxine query_on_thyroxine on_antithyroid_meds sick  \
2976    455   F            f                  f                   f    f   
5710  65511   M            f                  f                   f    f   
6392  65512   M            f                  f                   f    f   
8105  65526   F            f                  f                   f    f   

     pregnant thyroid_surgery I131_treatment query_hypothyroid  ... tumor  \
2976        f               f              f                 f  ...     f   
5710        f               f              f                 f  ...     f   
6392        f               f              f                 f  ...     f   
8105        f               f              f                 f  ...     f   

     hypopituitary psych   TSH   T3    TT4   T4U    FTI  TBG    target  
2976             f     f  1.10  2.0  118.0  1.13  104.0  NaN  negative  
5710             f     f  0.38  2.0  113.0  1.08  104.0  NaN  negative  
6392             f     f  0.03  2.8  112.0  0.84  133.0  NaN  negative  
8105             f     f  1.50  NaN  132.0  1.02  129.0  NaN  negative  

[4 rows x 23 columns]

mengubah nilai usia > 100 dengan null

In [355]:
df['age'] = np.where((df.age>100), np.nan, df.age)

In [356]:
df['age'] > 100

0       False
1       False
2       False
3       False
4       False
        ...  
9167    False
9168    False
9169    False
9170    False
9171    False
Name: age, Length: 9172, dtype: bool

Melakukan pengecekan pada data, untuk menemukan nilai mising value pada masing-masing kolom di datasets

In [357]:
df.isnull().sum()

age                       4
sex                     307
on_thyroxine              0
query_on_thyroxine        0
on_antithyroid_meds       0
sick                      0
pregnant                  0
thyroid_surgery           0
I131_treatment            0
query_hypothyroid         0
query_hyperthyroid        0
lithium                   0
goitre                    0
tumor                     0
hypopituitary             0
psych                     0
TSH                     842
T3                     2604
TT4                     442
T4U                     809
FTI                     802
TBG                    8823
target                 1626
dtype: int64

Berdasarkan hasil pengecekan datasets, terdapat 7 kolom yang mengandung nilai mising. antara lain sex 307 baris, TSH 842 baris,T3 2604, TT4 442, T4U 809, FTI 802, TBG 8823,  

mengingat mising value yang terdapat pada dataset sangat tidak relevan jika menyelesaikanya dengan cara yang konvensional, seperti nilai men, median, modus. pada pendekatan ini kami gunakan algortima KNN imputer untuk menyelesaikanya. setelah menemukan data mising pada colum selanjutnya melakukan seleksi terhadap kolom yang memiliki nilai mising.

In [358]:
from sklearn.impute import KNNImputer
df_col_miss = df.iloc[:, [1,16,17,18,19,20,21]]
df = df.iloc[:, 1:15]

In [359]:
df_col_miss

sex  TSH   T3    TT4   T4U   FTI   TBG
0      F  0.3  NaN    NaN   NaN   NaN   NaN
1      F  1.6  1.9  128.0   NaN   NaN   NaN
2      F  NaN  NaN    NaN   NaN   NaN  11.0
3      F  NaN  NaN    NaN   NaN   NaN  26.0
4      F  NaN  NaN    NaN   NaN   NaN  36.0
...   ..  ...  ...    ...   ...   ...   ...
9167   M  NaN  NaN   64.0  0.83  77.0   NaN
9168   M  NaN  NaN   91.0  0.92  99.0   NaN
9169   M  NaN  NaN  113.0  1.27  89.0   NaN
9170   F  NaN  NaN   75.0  0.85  88.0   NaN
9171   M  NaN  NaN   66.0  1.02  65.0   NaN

[9172 rows x 7 columns]

setelah melakukan pemilihan, perlu untuk melakukan penskalaan pada dataset dikarenakan algoritma KNNimputation megharuskan databernilai number. JIka melihat data terdapat nilai pada kolom sex masih menggunakankan tipe text, oleh karena itu pada kolom sex perlu melakukan convert dari tipe data text manjadi number.

In [360]:
def sex_miss(x):
    if x == 'F' : 
        return 2
    if x == 'M':
        return 1

In [361]:
col_sex = df_col_miss['sex'].apply(sex_miss)

menggabungkan kolom sex ke kolom df_col_miss


In [362]:
df_col_miss = df_col_miss.drop(['sex'], axis=1)

In [363]:
df_col_miss = pd.concat([col_sex, df_col_miss], axis=1)

In [364]:
df_col_miss

sex  TSH   T3    TT4   T4U   FTI   TBG
0     2.0  0.3  NaN    NaN   NaN   NaN   NaN
1     2.0  1.6  1.9  128.0   NaN   NaN   NaN
2     2.0  NaN  NaN    NaN   NaN   NaN  11.0
3     2.0  NaN  NaN    NaN   NaN   NaN  26.0
4     2.0  NaN  NaN    NaN   NaN   NaN  36.0
...   ...  ...  ...    ...   ...   ...   ...
9167  1.0  NaN  NaN   64.0  0.83  77.0   NaN
9168  1.0  NaN  NaN   91.0  0.92  99.0   NaN
9169  1.0  NaN  NaN  113.0  1.27  89.0   NaN
9170  2.0  NaN  NaN   75.0  0.85  88.0   NaN
9171  1.0  NaN  NaN   66.0  1.02  65.0   NaN

[9172 rows x 7 columns]

# Memperbaiki data Nan

In [365]:
mean = df_col_miss[["TSH","T3","TT4","T4U","FTI","TBG"]].mean()

In [366]:
df_col_miss = df_col_miss.replace(np.nan, mean)
df_col_miss

sex       TSH        T3         TT4       T4U         FTI        TBG
0     2.0  0.300000  1.970629  108.700305  0.976056  113.640746  29.870057
1     2.0  1.600000  1.900000  128.000000  0.976056  113.640746  29.870057
2     2.0  5.218403  1.970629  108.700305  0.976056  113.640746  11.000000
3     2.0  5.218403  1.970629  108.700305  0.976056  113.640746  26.000000
4     2.0  5.218403  1.970629  108.700305  0.976056  113.640746  36.000000
...   ...       ...       ...         ...       ...         ...        ...
9167  1.0  5.218403  1.970629   64.000000  0.830000   77.000000  29.870057
9168  1.0  5.218403  1.970629   91.000000  0.920000   99.000000  29.870057
9169  1.0  5.218403  1.970629  113.000000  1.270000   89.000000  29.870057
9170  2.0  5.218403  1.970629   75.000000  0.850000   88.000000  29.870057
9171  1.0  5.218403  1.970629   66.000000  1.020000   65.000000  29.870057

[9172 rows x 7 columns]

In [367]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
df_col_miss = pd.DataFrame(
    sc.fit_transform(df_col_miss), 
    columns=df_col_miss.columns
)

In [368]:
df_col_miss

sex       TSH        T3       TT4       T4U       FTI       TBG
0     1.0  0.000557  0.106999  0.178429  0.373174  0.127604  0.148925
1     1.0  0.003009  0.103064  0.210702  0.373174  0.127604  0.148925
2     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.054527
3     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.129565
4     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.179590
...   ...       ...       ...       ...       ...       ...       ...
9167  0.0  0.009837  0.106999  0.103679  0.305556  0.085948  0.148925
9168  0.0  0.009837  0.106999  0.148829  0.347222  0.110960  0.148925
9169  0.0  0.009837  0.106999  0.185619  0.509259  0.099591  0.148925
9170  1.0  0.009837  0.106999  0.122074  0.314815  0.098454  0.148925
9171  0.0  0.009837  0.106999  0.107023  0.393519  0.072306  0.148925

[9172 rows x 7 columns]

In [369]:
from sklearn.impute import KNNImputer
imputer  = KNNImputer(n_neighbors=5)
df_col_miss = pd.DataFrame(
    imputer.fit_transform(df_col_miss), columns=df_col_miss.columns
)

In [370]:
df_col_miss

sex       TSH        T3       TT4       T4U       FTI       TBG
0     1.0  0.000557  0.106999  0.178429  0.373174  0.127604  0.148925
1     1.0  0.003009  0.103064  0.210702  0.373174  0.127604  0.148925
2     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.054527
3     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.129565
4     1.0  0.009837  0.106999  0.178429  0.373174  0.127604  0.179590
...   ...       ...       ...       ...       ...       ...       ...
9167  0.0  0.009837  0.106999  0.103679  0.305556  0.085948  0.148925
9168  0.0  0.009837  0.106999  0.148829  0.347222  0.110960  0.148925
9169  0.0  0.009837  0.106999  0.185619  0.509259  0.099591  0.148925
9170  1.0  0.009837  0.106999  0.122074  0.314815  0.098454  0.148925
9171  0.0  0.009837  0.106999  0.107023  0.393519  0.072306  0.148925

[9172 rows x 7 columns]

# gabungkan kolom yang telah di fix mising ke data utama

In [371]:
df =df.drop(['sex'], axis=1)

In [372]:
df.head()

on_thyroxine query_on_thyroxine on_antithyroid_meds sick pregnant  \
0            f                  f                   f    f        f   
1            f                  f                   f    f        f   
2            f                  f                   f    f        f   
3            f                  f                   f    f        f   
4            f                  f                   f    f        f   

  thyroid_surgery I131_treatment query_hypothyroid query_hyperthyroid lithium  \
0               f              f                 t                  f       f   
1               f              f                 f                  f       f   
2               f              f                 f                  t       f   
3               f              f                 f                  f       f   
4               f              f                 f                  f       f   

  goitre tumor hypopituitary  
0      f     f             f  
1      f     f             f  
2      f     f             f  
3      f     f             f  
4      f     f             f

In [373]:
df = pd.concat([df, df_col_miss], axis=1)

In [374]:
df.head()

on_thyroxine query_on_thyroxine on_antithyroid_meds sick pregnant  \
0            f                  f                   f    f        f   
1            f                  f                   f    f        f   
2            f                  f                   f    f        f   
3            f                  f                   f    f        f   
4            f                  f                   f    f        f   

  thyroid_surgery I131_treatment query_hypothyroid query_hyperthyroid lithium  \
0               f              f                 t                  f       f   
1               f              f                 f                  f       f   
2               f              f                 f                  t       f   
3               f              f                 f                  f       f   
4               f              f                 f                  f       f   

  goitre tumor hypopituitary  sex       TSH        T3       TT4       T4U  \
0      f     f             f  1.0  0.000557  0.106999  0.178429  0.373174   
1      f     f             f  1.0  0.003009  0.103064  0.210702  0.373174   
2      f     f             f  1.0  0.009837  0.106999  0.178429  0.373174   
3      f     f             f  1.0  0.009837  0.106999  0.178429  0.373174   
4      f     f             f  1.0  0.009837  0.106999  0.178429  0.373174   

        FTI       TBG  
0  0.127604  0.148925  
1  0.127604  0.148925  
2  0.127604  0.054527  
3  0.127604  0.129565  
4  0.127604  0.179590

In [375]:
df.describe()

sex          TSH           T3          TT4          T4U  \
count  9172.000000  9172.000000  9172.000000  9172.000000  9172.000000   
mean      0.685041     0.009837     0.106999     0.178429     0.373174   
std       0.458514     0.043486     0.041843     0.061216     0.088574   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.001104     0.091922     0.143813     0.324074   
50%       1.000000     0.003009     0.106999     0.173913     0.373174   
75%       1.000000     0.006972     0.119777     0.204013     0.407407   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

               FTI          TBG  
count  9172.000000  9172.000000  
mean      0.127604     0.148925  
std       0.045126     0.020542  
min       0.000000     0.000000  
25%       0.106412     0.148925  
50%       0.125739     0.148925  
75%       0.141655     0.148925  
max       1.000000     1.000000